In [44]:
pip install catboost

     |████████████████████████████████| 76.1 MB 1.2 MB/s 


In [47]:
import pandas as pd
import numpy as np
import scipy.sparse as scp
import pickle
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score

In [55]:
model = CatBoostClassifier()
model.load_model("/content/drive/MyDrive/iVision_contest/catboost.cbm")

In [33]:
all_users = pd.read_csv('/content/drive/MyDrive/iVision_contest/01_all_users.csv')
all_users.contract_id.min(), all_users.contract_id.max(), len(pd.unique(all_users.contract_id))
events_log = pd.read_csv('/content/drive/MyDrive/iVision_contest/02_events_log.csv')
user_events_log = pd.read_csv('/content/drive/MyDrive/iVision_contest/03_lk_events_log.csv')
is_blocked = pd.read_csv('/content/drive/MyDrive/iVision_contest/04_is_blocked.csv')
test_df = pd.read_csv('/content/drive/MyDrive/iVision_contest/test.csv')
test_id = list(test_df.contract_id.unique())

In [34]:
z = set(all_users.contract_id.unique())
p1 = set(events_log.contract_id.unique())
p2 = set(user_events_log.contract_id.unique())
p3 = set(is_blocked.contract_id.unique())
t = set(test_df.contract_id.unique())

In [24]:
import joblib
vectorizer_1 = joblib.load('/content/drive/MyDrive/iVision_contest/vectroizer_1.pkl')
vectorizer_2 = joblib.load('/content/drive/MyDrive/iVision_contest/vectroizer_2.pkl')

In [25]:
wr_event_to_idx = {event_id : idx for idx, event_id in enumerate(events_log.event_type.unique())}
idx_to_wr_event = {i: event for event, i in wr_event_to_idx.items()}

event_to_idx = {event_id : idx for idx, event_id in enumerate(user_events_log.name.unique())}
idx_to_event = {i: event for event, i in event_to_idx.items()}

In [26]:
user_to_idx = {event_id : idx for idx, event_id in enumerate(test_id)}
idx_to_user = {i: event for event, i in user_to_idx.items()}

test_events_log = events_log.loc[events_log['contract_id'].isin(test_id)].copy()
test_events_log['contract_id'] = test_events_log['contract_id'].apply(lambda x: user_to_idx[x])
test_events_log['event_type'] = test_events_log['event_type'].apply(lambda x: wr_event_to_idx[x])

test_user_events_log = user_events_log.loc[user_events_log['contract_id'].isin(test_id)].copy()
test_user_events_log['contract_id'] = test_user_events_log['contract_id'].apply(lambda x: user_to_idx[x])
test_user_events_log['name'] = test_user_events_log['name'].apply(lambda x: event_to_idx[x])

test_events_log['const'] = np.ones(test_events_log.shape[0])
test_user_events_log['const'] = np.ones(test_user_events_log.shape[0])

matrica_1 = scp.coo_matrix(
        (
            test_events_log['const'],  # оценки пользователей
            (test_events_log['contract_id'], test_events_log['event_type'])  # id пользователей и id аниме, для которых известны оценки
        ), 
        shape=(len(test_id), len(wr_event_to_idx))  # размеры матрицы рейтингов
    ).todense().A

work_data = pd.DataFrame(data=matrica_1, columns=list(wr_event_to_idx.keys()))
work_data['contract_id'] = [idx_to_user[i] for i in range(len(test_id))]

In [27]:
unic = set()
st = ' '.join(["d"+str(i) for i in test_events_log.event_type.unique()])
res = [st]
for i in range(len(test_id)):
  user = test_events_log[test_events_log.contract_id == i]
  events = ' '.join(["d"+str(i) for i in list(sorted(user.event_type))])
  if len(events) == 0:
    events = '999'
  res.append(events)
  for i in events.split():
    unic.add(i)
X = vectorizer_1.transform(res)   # сделать просто fit
zz = pd.DataFrame(X.toarray()[1:], columns=vectorizer_1.get_feature_names_out())
zz['contract_id'] = [idx_to_user[i] for i in range(len(test_id))]
zz_2 = zz.copy()
zz_2.head() # only tf-idf
train_work_df = pd.merge(work_data, zz, on='contract_id') 

In [29]:
matrica_2 = scp.coo_matrix(
        (
            test_user_events_log['const'],  # оценки пользователей
            (test_user_events_log['contract_id'], test_user_events_log['name'])  # id пользователей и id аниме, для которых известны оценки
        ), 
        shape=(len(test_id), len(event_to_idx))  # размеры матрицы рейтингов
    ).todense().A
user_data = pd.DataFrame(data=matrica_2, columns=list(event_to_idx.keys()))
user_data['contract_id'] = [idx_to_user[i] for i in range(len(test_id))]
unic = set()
st = ' '.join(["u"+str(i) for i in test_user_events_log.name.unique()])
res = [st]
for i in range(len(test_id)):
  user = test_user_events_log[test_user_events_log.contract_id == i]
  events = ' '.join(["u"+str(i) for i in list(sorted(user.name))])
  if len(events) == 0:
    events = '111'
  res.append(events)
  for i in events.split():
    unic.add(i)
X = vectorizer_2.transform(res) 
zz = pd.DataFrame(X.toarray()[1:], columns=vectorizer_2.get_feature_names_out())
zz['contract_id'] = [idx_to_user[i] for i in range(len(test_id))]

In [30]:
work_data_2 = work_data.copy()
work_data_2.head()

,Добавление в Обращались с номеров,Обращение в службу заботы о клиентах,Выключение IPTV-пакета,Включение IPTV-пакета,"Включение услуги ""Ограничение функций ТВ""",Отключение услуги Аренда ТВ-приставки,Оборудование,Изменение истории,Продление действия тарифа,Предложение IPTV,Активация точки подключения на сутки,Смена тарифа,Удаление из Обращались с номеров,Смена пароля,Предложение ТВ,Внутреннее сообщение,Операция с mac-адресом,Закрепление сообщения,Перенос денежных средств,Изменение данных клиента,Возвращение клиента,Уведомления: подключение,Уведомления: подтверждение,Состояние клиентского оборудования,Смена схемы оплаты,"Включение услуги ""IPTV-Бонус""",Включена 20% скидка,Включение услуги Аренда ТВ-приставки,"Отключение услуги ""IPTV-Бонус""",Отключение скидки 20%,Предложение тарифа,Активация гарантированного платежа,Включение интернета на 20 минут,Изменение контактных телефонов,Внесение,Смена IP-адреса,Блокировка,Реестр договоров,Смена следующего тарифа,"Предложение акции ""Пригласи друга""",Предложение ТВ-пакета,Предложение автоплатежа,Активация ГП с IVR,Включение Бесплатной аренды,Замена оборудования,Удаление телефона из записи,Без границ бесплатно. Включение,Отключение скидки,Включена 30% скидка,Активация автоматического гарантированного платежа,Отключение услуги Автоплатёж,Расторжение,"Включена рассрочка за ""Wi-Fi маршрутизатор 2.4 ГГц - TP-LINK WR-850N или аналог""",Отмена гарантированного платежа,Манипуляции с оборудованием,Включена 10% скидка,Блокировка лицевого счета,Включение блокировки лицевого счета,Чек лист выполненных работ при регистрации,Разблокировка,Добавление IP-адреса,Удаление ip-адреса,Предложение расширенной гарантии на ТВ приставку,Без границ бесплатно. Отключение,Уведомления: отключение,Отключение скидки 10%,"Включение рассрочки за ""Подключение""",Окончание рассрочки за подключение,Предложение доп.гарантии на роутер,Предложение роутера,Тайный покупатель,Возвращение гарантированного платежа,"Включена рассрочка за ""IPTV-приставка""",Включение старых условий аренды,contract_id
0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1034
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1074
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1145
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1261
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1315


In [36]:
const = pd.Timestamp('2021-03-01 00:06:34')
const_2 = pd.Timestamp('2021-03-01 00:02:01')
def find_out_the_time_work(id):
  qq = test_events_log[test_events_log.contract_id == id]
  all_time = []
  for i in qq.event_date:
    time = pd.to_datetime(i, format='%Y-%m-%d %H:%M:%S.%f')
    #print(time)
    all_time.append(time)
  mi = (min(all_time)-const).days
  ma = (max(all_time)-const).days
  dif = ma-mi
  a = [i.days * 86400 +
       i.seconds for i in np.diff(np.array(all_time))]
  #print(sum(a) // len(all_time))
  return (min(all_time)-const).days, (max(all_time)-const).days, dif, sum(a) // len(all_time)

def find_out_the_time_user(id):
  qq = test_user_events_log[test_user_events_log.contract_id == id]
  all_time = []
  for i in qq.event_date:
    time = pd.to_datetime(i, format='%Y-%m-%d %H:%M:%S.%f')
    #print(time)
    all_time.append(time)
  mi = (min(all_time)-const_2).days
  ma = (max(all_time)-const_2).days
  dif = ma-mi
  a = [i.days * 86400 +
       i.seconds for i in np.diff(np.array(all_time))]
  #print(sum(a) // len(all_time))
  return (min(all_time)-const_2).days, (max(all_time)-const_2).days, dif, int(sum(a) // len(all_time))

In [37]:
work_data_2 = work_data.copy()
is_ampty_work = np.zeros(len(test_id))
is_ampty_user = np.zeros(len(test_id))
count_call_work = np.zeros(len(test_id))
count_call_user = np.zeros(len(test_id))
for i in range(len(test_id)):
  # есть ли какая то инфа
  if idx_to_user[i] not in p1:
    is_ampty_work[i] = 1
  else:
    qq = test_events_log[test_events_log.contract_id == i]
    count_call_work[i] = qq.shape[0]
  if idx_to_user[i] not in p2:
    is_ampty_user[i] = 1
  else:
    qq = test_user_events_log[test_user_events_log.contract_id == i]
    count_call_user[i] = qq.shape[0]
work_data_2['is_ampty_work'] = is_ampty_work
work_data_2['count_call_work'] = count_call_work
user_data['is_ampty_user'] = is_ampty_user
user_data['count_call_user'] = count_call_user

first_event_work = np.zeros(len(test_id))  # первое событие
first_event_user = np.zeros(len(test_id))
last_event_work = np.zeros(len(test_id))  # последнее событие
last_event_user = np.zeros(len(test_id))
difference_events_work = np.zeros(len(test_id))  # разность первого и последнего
difference_events_user = np.zeros(len(test_id))
mean_events_work = np.zeros(len(test_id))  # среднее разность всех событий
mean_events_user = np.zeros(len(test_id))
for i in range(len(test_id)):
  # есть ли какая то инфа
  if idx_to_user[i] in p1:
    first_event_work[i], last_event_work[i], difference_events_work[i], mean_events_work[i] = find_out_the_time_work(i)
  if idx_to_user[i] in p2:
    first_event_user[i], last_event_user[i], difference_events_user[i], mean_events_user[i] = find_out_the_time_user(i)

In [38]:
big_df_last1 = pd.merge(user_data, work_data_2, on='contract_id')
big_df_last1 = pd.merge(big_df_last1, all_users, on='contract_id').drop_duplicates()
big_df_last1['first_event_work'] = first_event_work
big_df_last1['first_event_user'] = first_event_user
big_df_last1['last_event_work'] = last_event_work
big_df_last1['last_event_user'] = last_event_user
big_df_last1['difference_events_work'] = difference_events_work
big_df_last1['difference_events_user'] = difference_events_user
big_df_last1['mean_events_work'] = mean_events_work
big_df_last1['mean_events_user'] = mean_events_user
big_df_last2 = pd.merge(zz_2, zz, on='contract_id')
big_df_last = pd.merge(big_df_last1, big_df_last2, on='contract_id')
big_df_last = big_df_last.drop(["contract_id"], axis=1)

In [39]:
big_df_last.iloc[:5]

,Информер ВК. Показ,Включение интернета на 20 минут_x,Показ информатора об изменении уведомлений через ВК платформу,Гарантированный платеж за деньги,Просмотр раздела Тайный покупатель,Включение интернета на 20 минут с IVR,Турбокнопка платно (1 час),Включение автоматической активации ГП,Отключение услуги Автоплатёж_x,Информер ВК. Закрытие,Виртуальное бессмертие. Включение,Виртуальное бессмертие. Использование,Без границ бесплатно. Отключение_x,Принято заявление и доп. соглашение по переезду,Смена пароля_x,Абонент оставил отзыв в Личном кабинете,Получение подарка в ЛК: сутки интернета на максимальной скорости,Получение подарка в ЛК: 1 месяц бесплатного просмотра IPTV,Включение IPTV-пакета_x,Выключение IPTV-пакета_x,Смена тарифа_x,Включение ограничения функций ТВ,Без границ платно. Отключение,Турбокнопка платно (1 сутки),Турбокнопка бесплатно,Без границ бесплатно. Включение_x,Без границ платно. Включение,Виртуальное бессмертие. Отключение,Тайный покупатель. Отправлена анкета,Бонус-скорость,Отключение ограничения функций ТВ,Изменение пакета HD. Закрытие лендинг-информера,Родительский контроль. Включение,Отключение управления услугами по паролю,Родительский контроль. Выключение,Восстановление пароля,Становление участником бонусной программы,Изменение схемы оплаты,Футболка Ситилинк,Отключение автоматической активации ГП,...,u16,u17,u18,u19,u2,u20,u21,u22,u23,u24,u25,u26,u27,u28,u29,u3,u30,u31,u32,u33,u34,u35,u36,u37,u38,u39,u4,u40,u41,u42,u43,u44,u45,u46,u47,u5,u6,u7,u8,u9
0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.74567,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
2,5.0,5.0,0.0,17.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.952121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.087446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
3,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.613539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0


In [40]:
big_df_last['contract_id'] = [idx_to_user[i] for i in range(len(test_id))]
we_dont_know = big_df_last[(big_df_last.is_ampty_user == 1) & (big_df_last.is_ampty_work == 1)]
we_know = big_df_last[(big_df_last.is_ampty_user == 0) | (big_df_last.is_ampty_work == 0)]
we_know

,Информер ВК. Показ,Включение интернета на 20 минут_x,Показ информатора об изменении уведомлений через ВК платформу,Гарантированный платеж за деньги,Просмотр раздела Тайный покупатель,Включение интернета на 20 минут с IVR,Турбокнопка платно (1 час),Включение автоматической активации ГП,Отключение услуги Автоплатёж_x,Информер ВК. Закрытие,Виртуальное бессмертие. Включение,Виртуальное бессмертие. Использование,Без границ бесплатно. Отключение_x,Принято заявление и доп. соглашение по переезду,Смена пароля_x,Абонент оставил отзыв в Личном кабинете,Получение подарка в ЛК: сутки интернета на максимальной скорости,Получение подарка в ЛК: 1 месяц бесплатного просмотра IPTV,Включение IPTV-пакета_x,Выключение IPTV-пакета_x,Смена тарифа_x,Включение ограничения функций ТВ,Без границ платно. Отключение,Турбокнопка платно (1 сутки),Турбокнопка бесплатно,Без границ бесплатно. Включение_x,Без границ платно. Включение,Виртуальное бессмертие. Отключение,Тайный покупатель. Отправлена анкета,Бонус-скорость,Отключение ограничения функций ТВ,Изменение пакета HD. Закрытие лендинг-информера,Родительский контроль. Включение,Отключение управления услугами по паролю,Родительский контроль. Выключение,Восстановление пароля,Становление участником бонусной программы,Изменение схемы оплаты,Футболка Ситилинк,Отключение автоматической активации ГП,...,u17,u18,u19,u2,u20,u21,u22,u23,u24,u25,u26,u27,u28,u29,u3,u30,u31,u32,u33,u34,u35,u36,u37,u38,u39,u4,u40,u41,u42,u43,u44,u45,u46,u47,u5,u6,u7,u8,u9,contract_id
0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.745670,0.0,0.0,0.0,0.0,1034
2,5.0,5.0,0.0,17.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.952121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.087446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1145
3,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.613539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1261
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1315
6,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,3.0,2.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.228996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.659667,0.0,0.0,0.0,0.0,274468
296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0

In [41]:
len(idx_to_user), len(test_id)

(300, 300)

In [49]:
predictions_1 = model.predict(we_know.drop(['contract_id'], axis=1))
predictions_1

array([0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 0])

In [50]:
predictions_2 = np.zeros(len(model.predict(we_dont_know.drop(['contract_id'], axis=1))))
predictions_2

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [51]:
we_know.contract_id
temp_1 = dict(zip(we_know.contract_id, predictions_1))
temp_2 = dict(zip(we_dont_know.contract_id, predictions_2))
x = {**temp_1, **temp_2}
result = pd.DataFrame({"contract_id": test_id,
                       "blocked": [x[i] for i in test_id]})

In [52]:
ttt = test_df.copy()
ttt['blocked'] = [int(x[i]) for i in ttt.contract_id]

In [53]:
ttt

,contract_id,blocked
0,1034,0
1,1074,0
2,1145,0
3,1261,0
4,1315,1
...,...,...
296,274468,1
297,274482,1
298,274486,1
299,274540,0


In [54]:
ttt.to_csv("submit.csv", index=False)

In [ ]:
model.save_model("catboost.cbm")